In [1]:
import pymysql
from functools import partial
import requests
import networkx as nx
import freeman as fm
from config import *
from datetime import datetime, timedelta
import matplotlib.pyplot as plt
import pandas as pd 

def run_db_query(connection, query, args=None):
    with connection.cursor() as cursor:
        print('Executando query:')
        cursor.execute(query, args)
        for result in cursor:
            print(result)

connection = pymysql.connect(
    host="db-restored.cqojuevfojvd.us-east-1.rds.amazonaws.com",
    user=myuser,
    password=mypass,
    database='twitch')


db = partial(run_db_query, connection)

In [2]:
def get_games(conn):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM game')
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
        
def get_links(conn):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM links')
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None

def get_links_from_game(conn,game):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM links WHERE links_game_name = %s',(game))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
        
        
# def get_username_from_link(conn,game):
#     with conn.cursor() as cursor:
#         cursor.execute(
#             '''SELECT streamer_display_name FROM streamer
#             INNER JOIN on 
#             WHERE streamer_id = %s''',(game))
#         res = cursor.fetchall()
#         if res:
#             return res
#         else:
#             return None
        
def get_username_from_id(conn,_id):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT streamer_display_name FROM streamer WHERE streamer_id = %s',(_id))
        res = cursor.fetchone()
        if res:
            return res
        else:
            return None
        
def get_streamer_id_from_game(conn,game_id):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT stream_user_id FROM twitch.stream WHERE stream_game_id = %s GROUP BY stream_user_id ;',(game_id))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
def get_total_viewer_count(conn,username):
    with conn.cursor() as cursor:
        cursor.execute("SELECT streamer_view_count FROM twitch.streamer WHERE streamer_display_name = %s",(username))
        res=cursor.fetchone()
        if res:
            return res
        else:
            return None
    
    
        
def get_streams_from_game(conn,time1,time2,game):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM stream WHERE stream_game_id = %s AND stream_request_time BETWEEN %s AND %s order by stream_viewer_count desc',(game, time1, time2))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None

def get_time(conn,time_1,time_2,game_id):
    #print(time_1,time_2,game_id)
    r = get_streams_from_game(conn,time_1,time_2,game_id)
    #print(r)
    sizes = {}
    if r:
        for node in r:
            sizes[node[2]] = node[6]
    return sizes

def get_progression(conn,start_date,end_date,game_id):
    lista = []
    data = start_date
    while(data < end_date):
        lista.append(get_time(conn,str(data),str(data+timedelta(minutes=10)),game_id))
        data += timedelta(minutes=30)
    return lista

def list_streamer_time(time1,time2,game):
    totalview=0
    streams = get_streams_from_game(connection,time1,time2,game)
    Slist=[]
    if streams:
        for a in streams:
            totalview=totalview+a[6]
            Slist.append((a[2],a[6],a[1],a[8])) # user name, viewer count, user id, request time
    return Slist,totalview

def filter_value( someList, value ):
    for x, y,z,w in someList:
        if x == value :
            yield x,y
            

def get_leavers(list1,list2):
    leavers=[]
    for i in list1[0:30]:
        result=list( filter_value(list2, i[0]))
        if result==[]:
            leavers.append([i[0],i[2]])
    return leavers

def get_links_from_to(conn,from_id,to_id):
    with conn.cursor() as cursor:
        cursor.execute(
            'SELECT * FROM links WHERE from_id = %s AND to_id = %s',(from_id, to_id))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None
        
def links_dict(conn,game_name):
    with conn.cursor() as cursor:
        cursor.execute(
            '''
            SELECT st1.streamer_display_name, st2.streamer_display_name FROM links
            INNER JOIN streamer st1 ON links.from_id = st1.streamer_id
            INNER JOIN streamer st2 ON links.to_id = st2.streamer_id
            WHERE links_game_name LIKE %s LIMIT 8000
            ''',(game_name))
        res = cursor.fetchall()
        if res:
            return res
        else:
            return None

def segue(name1,name2,dicti):
    a = [(x,y) for x,y in dicti if x ==name1 and y == name2]
    if a != []:
        return True
    
    return False



In [4]:
def generate_dataframe(Antes,Depois,dicti):
    # Antes eh [0] lista de sizes e [1] eh o total view de todo mundo somado
    # Depois eh [0] lista de sizes e [1] eh o total view de todo mundo somado
    
    df=[]
    gt = fm.Graph(nx.DiGraph()) # Grafo
    leavers=get_leavers(Antes[0],Depois[0]) # Lista de sizes
    for node in Antes[0]:
        #print(node[0])
        for node2 in Antes[0]:
            #print(segue(node[0],node2[0],dicti))
            if segue(node[0],node2[0],dicti):
                gt.add_edges_from([(node[0], node2[0])])
    #return gt
        
    
    if leavers !=[]:
        print('SAIU')
        print("Leavers",leavers)
        for i in Depois[0]:
            a=[tup for tup in Antes[0] if tup[0] == i[0]]  # Checa se o cara ta no Depois
            if a!=[]:
                
                #print(leavers)
                #print(a[0])
                data=[]
                #print(a,i)
                data.append(i[0])
                minDist = 49
                
                # Calcula percentagem delta
                percentAntes= (a[0][1]/Antes[1])*100 # a01 eh view da stream
                #print("Porcentagem antes ",percentAntes)
                percentDepois= (i[1]/Depois[1])*100 # i1 view dps
                #print("Porcentagem depois ",percentDepois)
                difBruta=i[1]-a[0][1] # Diferenca bruta
                difPercent=(percentDepois-percentAntes)
                data.append(difBruta)
                data.append(difPercent)
                
                # Pega os viwers totais
                try:
                    TotalViewer=get_total_viewer_count(connection,i[0])[0]
                except:
                    print(i,"Deu Erro De TotalViewer")
                    TotalViewer=0
                    
                data.append(a[0][1]) # Instant ViewCount
                data.append(TotalViewer)
                
                
                # Checa se os leavers seguiam ele
                count_leaver_that_follow = 0
                for leaver in leavers:
                    #print(leaver[1],a[0][2])
                    #links = get_links_from_to(connection, leaver[1], a[0][2]) # Checa se o leaver segue o user
                    links = segue(leaver[0],a[0][0],dicti)
                    if (links):
                        print("Usuario "+leaver[0]+" saiu. E seguia o "+a[0][0])
                        count_leaver_that_follow+=1
                    try:
                        dist = nx.shortest_path_length(gt.reverse(copy=False),source=leaver[0],target=a[0][0])
                        #print(dist)
                        if minDist is None or dist < minDist:
                            minDist = dist
                    except:
                        #print("N tem path")
                        pass
                    

                ## Distancia
                #print(leaver[],a[0][2])
                #print(nx.shortest_path_length(gt,source=,target=a[0][2]))
                        
                data.append(count_leaver_that_follow)
                data.append(i[3])
                data.append(minDist)
                df.append(data)
            #break
    return df, gt


def get_progression(start_date,end_date,game_id,dicti):
    DBase=[]
    data = start_date
    safedata=start_date+timedelta(minutes=10)
    while(data < end_date):
        s1=list_streamer_time(data,safedata,game_id)
        s2=list_streamer_time(data+timedelta(minutes=30),safedata+timedelta(minutes=30),game_id)
        d,g=generate_dataframe(s1,s2,dicti)
        DBase += d
        data += timedelta(minutes=30)
        safedata+=timedelta(minutes=30)
    print(DBase)
    df = pd.DataFrame(DBase, columns = ['UserName', 'DeltaAbsolute','DeltaPercentage',"ViewCount","TotalViewerCount","LeaversFollowers","RequestTime","Distance"])
    print("Acaaabou!")
    return df

In [6]:
dict_cs = links_dict(connection, "Counter-Strike: Global Offensive")
cs_id = "32399"

time1 = datetime(2019, 11, 11, 17, 0) # 2019-11-11 17:00:02
time2 = datetime(2019, 11, 16, 4,30) # 2019-11-16 04:45:07

cs = get_progression(time1,time2,cs_id,dict_cs)
cs.to_csv("cs.csv")
cs

SAIU
Leavers [['uccleague2', 89349239], ['OMENbyHPEurope', 88623939], ['EFEUYGAC', 103237625]]
Usuario uccleague2 saiu. E seguia o UCCleague
SAIU
Leavers [['UCCleague', 75910196]]
Usuario UCCleague saiu. E seguia o uccstudio5
Usuario UCCleague saiu. E seguia o uccstudio3
Usuario UCCleague saiu. E seguia o uccstudio4
SAIU
Leavers [['IzakOOO', 36717908], ['maleK_CSGO', 85034121]]
Usuario IzakOOO saiu. E seguia o esl_csgo_pl
Usuario IzakOOO saiu. E seguia o PAGO3
Usuario IzakOOO saiu. E seguia o inet_saju
SAIU
Leavers [['forsen', 22484632], ['valentineche', 139651213], ['Hiko', 26991127], ['jonzherka', 407881598]]
Usuario Hiko saiu. E seguia o ESL_CSGO
Usuario valentineche saiu. E seguia o cheatbanned
SAIU
Leavers [['yznb', 31582993], ['s1mpleiw', 472196095], ['aynisinemalar', 89408007]]
Usuario yznb saiu. E seguia o pashaBiceps
Usuario yznb saiu. E seguia o GeT_RiGhT
Usuario yznb saiu. E seguia o ESL_CSGO
SAIU
Leavers [['GeT_RiGhT', 38024128], ['MarckozHD', 32915274], ['KalashnikovMediaT

SAIU
Leavers [['FPSThailand', 38539112], ['m0xyy', 69012069], ['FPSThailand2', 143093008], ['1mpala', 122854486], ['s1mplerkd', 472196918], ['viperdemon', 39150803], ['zoommo_', 239012683]]
SAIU
Leavers [['zonixxcs', 52351531], ['冠賢', 31673401], ['s1mplesnn', 472196893], ['um_league', 268731435], ['ThunderMediaTV', 185126137], ['uccstudio', 122590578], ['headshinsky', 40947658], ['KXNStudio', 146473071]]
SAIU
Leavers [['mimimimichaela', 48289225], ['SquishyMuffinz', 90018770], ['Esports_Assoc', 198791562]]
SAIU
Leavers [['Trainwreckstv', 71190292], ['faceit__csgo', 472359438], ['Dhalucard', 16064695], ['yznb', 31582993], ['CyrusTWO', 41108115], ['RachelR_csgo', 104259136], ['mch_AGG', 75891532], ['KalashnikovMediaTV_2', 227786273], ['2SekundovyMato', 89751834], ['yaNgile', 76466180], ['卡薩夫', 31966894], ['1mpala', 122854486], ['inet_saju', 42080143], ['FURIAtv', 178500702], ['nookyyy', 24943033], ['CSRfps', 131210325], ['bladeito', 72602541]]
Usuario mch_AGG saiu. E seguia o gaules
Usua

SAIU
Leavers [['s1mplezsg', 472197577], ['True_Schu', 79135520]]
SAIU
Leavers [['s1mpletan', 472197579], ['Jynx', 103483444], ['igorevich', 102018894]]
SAIU
Leavers [['faaceitt', 472476508], ['冠賢', 31673401], ['s1mplekug', 472197472], ['viperdemon', 39150803], ['Sweezy', 47603866]]
SAIU
Leavers [['Laski', 39205266], ['navioot', 64788017], ['no_onee', 83194824], ['Spamned', 168823131]]
SAIU
Leavers [['CSRuHub', 116311210], ['WARDELLCSGO', 100182904], ['esbcsgo', 471972656]]
SAIU
Leavers [['faceit_cs_go', 472479993], ['s1mpletrz', 472197521], ['KRLAN', 129289579], ['Nozadah', 126695000], ['MUITOls', 43919115], ['ChillBoy4ik', 427385340], ['headon', 108948303]]
Usuario KRLAN saiu. E seguia o ESL_CSGO
SAIU
Leavers [['WARDELLCSGO', 100182904]]
SAIU
Leavers [['QRUSHcsgo', 104435562]]
SAIU
Leavers [['hardgamechannel', 153353959], ['SquishyMuffinz', 90018770], ['cheatbanned', 42049440], ['s1mplemzv', 472197439], ['tenshixgod', 75521067]]
SAIU
Leavers [['BezGMO', 193972944], ['Haraw', 28269648]

SAIU
Leavers [['lucasteles2', 51993057], ['LyndonFPS', 55433748], ['sektorlag', 247091743], ['Vitokng', 102610592], ['LeXCS', 39173262]]
SAIU
Leavers [['jdm64', 36611237], ['frostezor', 45951681]]
SAIU
Leavers [['navioot', 64788017], ['sisi2kek', 151141374], ['zaul_csgo', 471953949], ['SovetnikTV', 217894879]]
SAIU
Leavers [['Dareon', 130268575]]
SAIU
Leavers [['rootthegamer', 104781313], ['homeacademy3', 166633383], ['jeemzz', 105218968], ['Sweezy', 47603866], ['tenshixgod', 75521067], ['FURIAtv', 178500702], ['KRLAN', 129289579], ['CSRfps', 131210325], ['fer', 71167031], ['murilo_RT', 168327867], ['telkofftv', 194505685], ['headshinsky', 40947658], ['ChillBoy4ik', 427385340], ['MUITOls', 43919115], ['GALLERIA_GM', 168877278], ['d3met', 144760117]]
Usuario KRLAN saiu. E seguia o ESL_CSGO
Usuario CSRfps saiu. E seguia o gaules
Usuario murilo_RT saiu. E seguia o gaules
Usuario CSRfps saiu. E seguia o mch_AGG
SAIU
Leavers [['GamerAidAUS', 472294891]]
SAIU
Leavers [['ninoti', 472492136], 

SAIU
Leavers [['s1mplegng', 472699653], ['s1mplegng', 472699653], ['s1mplegng', 472699653], ['elizah_csgo', 472774883]]
SAIU
Leavers [['CSRuHub', 116311210], ['Mibrtv', 4794831], ['s1mplebks', 472699682]]
SAIU
Leavers [['s1mpleugn', 472699687], ['OfficialAndyPyro', 46021592]]
SAIU
Leavers [['s1mplecqk', 472699724], ['godasaf', 472728535], ['FPSThailand', 38539112], ['INS1DE1337', 147118402]]
Usuario INS1DE1337 saiu. E seguia o dmitriifiv
SAIU
Leavers [['s1mpleaik', 472699752], ['Sweezy', 47603866], ['volx', 94101038], ['d3met', 144760117]]
SAIU
Leavers [['s1mpleisj', 472699732], ['qqcscs', 132190230], ['BolaoHu3', 102210281]]
SAIU
Leavers [['godasaf', 472728535]]
SAIU
Leavers [['s1mpleagi', 472699736], ['Laski', 39205266], ['kawemo', 472652853], ['WARDELLCSGO', 100182904]]
SAIU
Leavers [['uccstudio5', 132250791], ['faceit_giveaway_day', 472801851], ['jeemzz', 105218968], ['Sweezy', 47603866]]
SAIU
Leavers [['homeacademy3', 166633383], ['tenshixgod', 75521067]]
SAIU
Leavers [['shoxieJES

,UserName,DeltaAbsolute,DeltaPercentage,ViewCount,TotalViewerCount,LeaversFollowers,RequestTime,Distance
0,forsen,-465,3.370713,10767,119785565,0,2019-11-11 17:30:15,49
1,esl_csgo_pl,-2356,-2.631381,6379,44760959,0,2019-11-11 17:30:15,49
2,IzakOOO,-18,1.074853,2725,146478203,0,2019-11-11 17:30:15,49
3,gaules,-75,0.845765,2475,78643635,0,2019-11-11 17:30:15,49
4,ESL_CSGO,323,1.467689,1831,403942065,0,2019-11-11 17:30:15,49
5,PAGO3,-106,0.595962,2033,29959694,0,2019-11-11 17:30:15,49
6,uccstudio5,1180,2.857470,568,6068830,0,2019-11-11 17:30:15,49
7,fl0m,92,0.862150,1607,37289665,0,2019-11-11 17:30:16,49
8,valentineche,201,0.961029,1256,4475115,0,2019-11-11 17:30:16,49
9,龜狗,0,0.567063,1386,16385518,0,2019-11-11 17:30:16,49


In [7]:
dict_lol = links_dict(connection, "League of Legends")
lol_id = "21779"

time1 = datetime(2019, 11, 11, 17, 0) # 2019-11-11 17:00:02
time2 = datetime(2019, 11, 16, 4,30) # 2019-11-16 04:45:07

lol = get_progression(time1,time2,lol_id,dict_lol)
lol.to_csv("lol.csv")
lol

SAIU
Leavers [['pankylol', 82388424], ['AzunyanOzge', 70818237], ['SoloRenektonOnly', 30227322]]
Usuario SoloRenektonOnly saiu. E seguia o TFBlade
Usuario SoloRenektonOnly saiu. E seguia o IWillDominate
Usuario SoloRenektonOnly saiu. E seguia o Shiphtur
Usuario SoloRenektonOnly saiu. E seguia o imaqtpie
Usuario pankylol saiu. E seguia o lol_Khan
Usuario pankylol saiu. E seguia o ThaldrinLol
Usuario AzunyanOzge saiu. E seguia o ThaldrinLol
Usuario pankylol saiu. E seguia o FabulousN
Usuario SoloRenektonOnly saiu. E seguia o PinkWardlol
SAIU
Leavers [['Faker', 43691], ['얍얍', 103861159]]
Usuario 얍얍 saiu. E seguia o 한동숙
Usuario 얍얍 saiu. E seguia o 서새봄냥
Usuario 얍얍 saiu. E seguia o 인간젤리
Usuario 얍얍 saiu. E seguia o 루시아
Usuario 얍얍 saiu. E seguia o 박옥자누나
Usuario 얍얍 saiu. E seguia o Terenas
SAIU
Leavers [['NoWay4u_Sir', 85397463], ['IWillDominate', 25653002], ['테디이', 152714599], ['スタンミ', 37323664], ['kev1nTV', 48919437]]
Usuario IWillDominate saiu. E seguia o TFBlade
Usuario IWillDominate saiu. 

Usuario Kamet0 saiu. E seguia o Pobelter
Usuario 서새봄냥 saiu. E seguia o 강소연
Usuario Froggen saiu. E seguia o Becca
Usuario Kamet0 saiu. E seguia o OgamingLoL
Usuario Kamet0 saiu. E seguia o Taourrr1030
SAIU
Leavers [['Solaaaa', 44864184], ['Cabech', 67493484]]
Usuario Cabech saiu. E seguia o coscu
Usuario Cabech saiu. E seguia o DuendePablo
Usuario Cabech saiu. E seguia o Rojankhzxr
SAIU
Leavers [['robi', 47862305], ['Pun1shers_TV', 67970187], ['Bustin', 73516344], ['らいじん', 45839490], ['kutcherlol', 65757403]]
Usuario Pun1shers_TV saiu. E seguia o loltyler1
Usuario Bustin saiu. E seguia o loltyler1
Usuario らいじん saiu. E seguia o boxbox
Usuario Pun1shers_TV saiu. E seguia o Trick2g
Usuario Bustin saiu. E seguia o Trick2g
Usuario らいじん saiu. E seguia o Trick2g
Usuario Pun1shers_TV saiu. E seguia o tarzaned
Usuario らいじん saiu. E seguia o たぬき忍者
Usuario Bustin saiu. E seguia o STPeach
Usuario robi saiu. E seguia o OgamingLoL
Usuario Bustin saiu. E seguia o foggedftw2
SAIU
Leavers [['yeTz', 2768

Usuario Voyboy saiu. E seguia o Becca
Usuario leplol saiu. E seguia o Lannik1
SAIU
Leavers [['NeVeR_LosEs', 29722828], ['TobiasFate', 91137296]]
Usuario TobiasFate saiu. E seguia o loltyler1
Usuario NeVeR_LosEs saiu. E seguia o 接接
Usuario NeVeR_LosEs saiu. E seguia o 狂暴小建
SAIU
Leavers [['かべお', 77430846], ['Becca', 35824977]]
Usuario Becca saiu. E seguia o loltyler1
Usuario Becca saiu. E seguia o C9Sneaky
Usuario Becca saiu. E seguia o boxbox
Usuario Becca saiu. E seguia o Aphromoo
Usuario かべお saiu. E seguia o しゃるる
Usuario Becca saiu. E seguia o keanelol
Usuario Becca saiu. E seguia o Cowsep
SAIU
Leavers [['Aphromoo', 21673391], ['KarasMai', 118241089], ['ORengaar', 54440664]]
Usuario Aphromoo saiu. E seguia o loltyler1
Usuario Aphromoo saiu. E seguia o C9Sneaky
Usuario ORengaar saiu. E seguia o gratis150ml
Usuario Aphromoo saiu. E seguia o stunt
Usuario KarasMai saiu. E seguia o Gripex90
Usuario ORengaar saiu. E seguia o ClovaoBengala
Usuario KarasMai saiu. E seguia o Cowsep
SAIU
Leave

Usuario Werlyb saiu. E seguia o LVPes
Usuario Werlyb saiu. E seguia o ElmiilloR
Usuario kev1nTV saiu. E seguia o Broeki1
Usuario Solary saiu. E seguia o Kamet0
Usuario Skyyart saiu. E seguia o Kamet0
Usuario Solary saiu. E seguia o Corobizar
Usuario Skyyart saiu. E seguia o Corobizar
Usuario Solary saiu. E seguia o robi
SAIU
Leavers [['Broeki1', 41372534], ['Kamet0', 27115917], ['AdamKissAK', 66996733]]
Usuario AdamKissAK saiu. E seguia o TFBlade
Usuario Kamet0 saiu. E seguia o imaqtpie
Usuario Kamet0 saiu. E seguia o Shiphtur
Usuario AdamKissAK saiu. E seguia o Shiphtur
Usuario Kamet0 saiu. E seguia o Aphromoo
Usuario AdamKissAK saiu. E seguia o Aphromoo
Usuario Kamet0 saiu. E seguia o Trick2g
Usuario AdamKissAK saiu. E seguia o Trick2g
Usuario Kamet0 saiu. E seguia o Froggen
Usuario AdamKissAK saiu. E seguia o Froggen
Usuario Kamet0 saiu. E seguia o Corobizar
Usuario Kamet0 saiu. E seguia o Locodoco
SAIU
Leavers [['TFBlade', 59308271], ['SoloRenektonOnly', 30227322], ['Opat04', 24383

Usuario Solaaaa saiu. E seguia o xFSN_Saber
Usuario jukes saiu. E seguia o 1Adrianaries1
Usuario jukes saiu. E seguia o pijack11
Usuario jukes saiu. E seguia o Cowsep
Usuario Trick2g saiu. E seguia o Cowsep
Usuario Solaaaa saiu. E seguia o Cowsep
Usuario jukes saiu. E seguia o Dekar173
SAIU
Leavers [['ElmiilloR', 44880944], ['Taourrr1030', 50910019], ['재슥짱', 194230187], ['kutcherlol', 65757403], ['STPeach', 100484450], ['rhobalas_lol', 151874024]]
Usuario ElmiilloR saiu. E seguia o loltyler1
Usuario rhobalas_lol saiu. E seguia o loltyler1
Usuario ElmiilloR saiu. E seguia o Aphromoo
Usuario ElmiilloR saiu. E seguia o Nightblue3
Usuario STPeach saiu. E seguia o Nightblue3
Usuario ElmiilloR saiu. E seguia o TFBlade
Usuario STPeach saiu. E seguia o TFBlade
Usuario rhobalas_lol saiu. E seguia o TFBlade
Usuario ElmiilloR saiu. E seguia o Voyboy
Usuario ElmiilloR saiu. E seguia o coscu
Usuario ElmiilloR saiu. E seguia o tarzaned
Usuario ElmiilloR saiu. E seguia o Pun1shers_TV
Usuario Taourrr1

...

Usuario Akytio saiu. E seguia o Solary
Usuario Wingsofdeath saiu. E seguia o SirhcEz
Usuario RandomBruceTV saiu. E seguia o d4mm_
Usuario Akytio saiu. E seguia o OgamingLoL
Usuario Wingsofdeath saiu. E seguia o NickiTaylor
Usuario RandomBruceTV saiu. E seguia o ProxyGinger
SAIU
Leavers [['leplol', 58351197], ['NiteNightKid', 48946894], ['Xigbaer', 44921835]]
Usuario NiteNightKid saiu. E seguia o C9Sneaky
Usuario Xigbaer saiu. E seguia o C9Sneaky
Usuario NiteNightKid saiu. E seguia o KarasMai
SAIU
Leavers [['이초홍', 138094321], ['루시아', 113164795]]
Usuario 루시아 saiu. E seguia o 이선생84
SAIU
Leavers [['C9Sneaky', 24538518], ['Shaunz', 43102846], ['CoreJJ', 74887420]]
Usuario Shaunz saiu. E seguia o Solary
Usuario Shaunz saiu. E seguia o GoBGG
Usuario Shaunz saiu. E seguia o OgamingLoL
SAIU
Leavers [['매드라이프_', 146584003], ['萌萌小蝶', 67021979], ['Closer', 160677372], ['TESLLive', 22147412], ['랄로', 29654002]]
Usuario 매드라이프_ saiu. E seguia o 앰비션_
Usuario 매드라이프_ saiu. E seguia o Solary
Usuario 매드라이프_

Usuario 1Adrianaries1 saiu. E seguia o ElmiilloR
Usuario 1Adrianaries1 saiu. E seguia o boxbox
Usuario Solary saiu. E seguia o Kamet0
Usuario 1Adrianaries1 saiu. E seguia o tarzaned
Usuario Solary saiu. E seguia o IFresh_HD
Usuario Solary saiu. E seguia o OgamingLoL
Usuario 1Adrianaries1 saiu. E seguia o STPeach
Usuario 1Adrianaries1 saiu. E seguia o IKeepItTaco
Usuario 1Adrianaries1 saiu. E seguia o Dekar173
SAIU
Leavers [['brtt', 31497918], ['AdamKissAK', 66996733], ['xnapycz', 27370971], ['pimpimenta', 102346837]]
Usuario AdamKissAK saiu. E seguia o loltyler1
Usuario pimpimenta saiu. E seguia o loltyler1
Usuario AdamKissAK saiu. E seguia o Yassuo
Usuario pimpimenta saiu. E seguia o Yassuo
Usuario brtt saiu. E seguia o jukes
Usuario pimpimenta saiu. E seguia o jukes
Usuario AdamKissAK saiu. E seguia o Shiphtur
Usuario AdamKissAK saiu. E seguia o boxbox
Usuario xnapycz saiu. E seguia o Locodoco
Usuario pimpimenta saiu. E seguia o STPeach
Usuario xnapycz saiu. E seguia o LVPes
SAIU
Lea

Usuario LoLGeranimo saiu. E seguia o loltyler1
SAIU
Leavers [['loltyler1', 51496027]]
SAIU
Leavers [['이선생84', 68771419]]
SAIU
Leavers [['LynxCerezLoL', 161882527], ['untara', 158271876]]
SAIU
Leavers [['yvonnie', 45184940], ['루밍쨩', 145143869]]
SAIU
Leavers [['牛牛妹', 46624156]]
SAIU
Leavers [['leylunehar', 63198655], ['Wingsofdeath', 30171560]]
Usuario Wingsofdeath saiu. E seguia o loltyler1
Usuario Wingsofdeath saiu. E seguia o C9Sneaky
Usuario Wingsofdeath saiu. E seguia o boxbox
Usuario Wingsofdeath saiu. E seguia o Broxah
Usuario Wingsofdeath saiu. E seguia o hashinshin
Usuario Wingsofdeath saiu. E seguia o SirhcEz
Usuario Wingsofdeath saiu. E seguia o midbeast
Usuario Wingsofdeath saiu. E seguia o Cowsep
Usuario Wingsofdeath saiu. E seguia o LoLGeranimo
SAIU
Leavers [['C9Sneaky', 24538518], ['Broxah', 43539324], ['지수소녀', 102619512], ['yamatosdeath1', 188887325]]
Usuario C9Sneaky saiu. E seguia o loltyler1
Usuario Broxah saiu. E seguia o loltyler1
Usuario Broxah saiu. E seguia o midb

Usuario TFBlade saiu. E seguia o imaqtpie
Usuario TFBlade saiu. E seguia o hashinshin
Usuario TFBlade saiu. E seguia o LLStylish
Usuario pankylol saiu. E seguia o ThaldrinLol
Usuario TFBlade saiu. E seguia o SoloRenektonOnly
Usuario TFBlade saiu. E seguia o Freezecz
Usuario pankylol saiu. E seguia o Crystal_LoL
Usuario TFBlade saiu. E seguia o Katerino
SAIU
Leavers [['gratis150ml', 52203144], ['kev1nTV', 48919437]]
Usuario gratis150ml saiu. E seguia o loltyler1
Usuario gratis150ml saiu. E seguia o jovirone
Usuario gratis150ml saiu. E seguia o Shiphtur
Usuario gratis150ml saiu. E seguia o imaqtpie
Usuario gratis150ml saiu. E seguia o hashinshin
Usuario gratis150ml saiu. E seguia o PinkWardlol
SAIU
Leavers [['Elwind', 71761252], ['SoloRenektonOnly', 30227322], ['실프_', 138009898], ['Juliostito', 35648959], ['Becca', 35824977]]
Usuario SoloRenektonOnly saiu. E seguia o loltyler1
Usuario Juliostito saiu. E seguia o loltyler1
Usuario Becca saiu. E seguia o loltyler1
Usuario SoloRenektonOnly 

,UserName,DeltaAbsolute,DeltaPercentage,ViewCount,TotalViewerCount,LeaversFollowers,RequestTime,Distance
0,Faker,-2081,-0.404500,41353,66177341,0,2019-11-11 17:30:02,49
1,한동숙,-273,0.251746,20103,50998396,0,2019-11-11 17:30:02,49
2,NoWay4u_Sir,-59,0.180010,10393,38866827,0,2019-11-11 17:30:02,49
3,얍얍,143,0.287557,9667,55118201,0,2019-11-11 17:30:02,49
4,TFBlade,1269,0.935057,7932,37547888,1,2019-11-11 17:30:02,49
5,서새봄냥,-571,-0.201111,7007,58053811,0,2019-11-11 17:30:02,49
6,IWillDominate,739,0.546207,4700,52797710,1,2019-11-11 17:30:02,49
7,Solary,-1715,-0.898330,6874,87110241,0,2019-11-11 17:30:03,49
8,clid,1185,0.779153,2880,957268,0,2019-11-11 17:30:03,49
9,jovirone,-323,-0.107194,4280,56151830,0,2019-11-11 17:30:03,49


In [8]:
dict_jc = links_dict(connection, "Just Chatting")
jc_id = "509658"

time1 = datetime(2019, 11, 11, 17, 0) # 2019-11-11 17:00:02
time2 = datetime(2019, 11, 16, 4,30) # 2019-11-16 04:45:07

jc = get_progression(time1,time2,jc_id,dict_jc)
jc.to_csv("jc.csv")
jc

SAIU
Leavers [['우왁굳', 49045679], ['ilMasseo', 55933037], ['Dekarldent', 137907801], ['웁_게임방송', 137753119], ['CyrusTWO', 41108115], ['CmdKrieger', 27751942], ['Sconza', 428069737], ['마뫄', 161982841], ['Deujna', 42541814], ['LeyaGornaya', 280805917], ['GENSYXA', 81623587]]
Usuario Deujna saiu. E seguia o KingRichard
Usuario 마뫄 saiu. E seguia o 견자희
Usuario ilMasseo saiu. E seguia o Homyatol
Usuario Dekarldent saiu. E seguia o Staiy
Usuario CmdKrieger saiu. E seguia o Staiy
Usuario GENSYXA saiu. E seguia o AhriNyan
Usuario 마뫄 saiu. E seguia o 악녀
Usuario LeyaGornaya saiu. E seguia o jamclub
SAIU
Leavers [['alanzoka', 38244180], ['KingRichard', 66691674], ['H2P_Gucio', 36954803], ['蝦愛橘子', 25202416], ['Kandyland', 57865494], ['segall', 35856714], ['Ko0416', 100438963], ['saniyekizismidegildir', 463836611], ['jonzherka', 407881598]]
Usuario segall saiu. E seguia o bratishkinoff
Usuario Kandyland saiu. E seguia o NymN
Usuario segall saiu. E seguia o Lasqa
Usuario Kandyland saiu. E seguia o djar

Usuario xQcOW saiu. E seguia o Mizkif
Usuario DevinNash saiu. E seguia o HasanAbi
Usuario DevinNash saiu. E seguia o TheSushiDragon
Usuario SkipNhO saiu. E seguia o pescocofino
Usuario SkipNhO saiu. E seguia o theRealShooKon3
Usuario DevinNash saiu. E seguia o Hitch
Usuario SkipNhO saiu. E seguia o PoliciaEmAcao
SAIU
Leavers [['RajjPatel', 40197643], ['Mizkif', 94753024], ['Amouranth', 125387632], ['HotTea', 250350524], ['MrQuarate', 89283616], ['ADouMy_NooB', 142871522], ['kamikpro', 49855925]]
Usuario RajjPatel saiu. E seguia o Kitboga
Usuario RajjPatel saiu. E seguia o ludwig
Usuario RajjPatel saiu. E seguia o HasanAbi
Usuario MrQuarate saiu. E seguia o TheSushiDragon
Usuario RajjPatel saiu. E seguia o STPeach
Usuario Amouranth saiu. E seguia o STPeach
Usuario RajjPatel saiu. E seguia o theRealShooKon3
Usuario RajjPatel saiu. E seguia o BruceDropEmOff
Usuario RajjPatel saiu. E seguia o SeriousGaming
Usuario RajjPatel saiu. E seguia o Hitch
Usuario RajjPatel saiu. E seguia o Blondiew

SAIU
Leavers [['Elwind', 71761252], ['segall', 35856714], ['InvaderVie', 198815529], ['芝衣芝衣', 205058897]]
SAIU
Leavers [['MontanaBlack88', 45044816], ['modestal', 112619759], ['knekro', 152633332], ['Homyatol', 21167655], ['Vitecp', 17261176], ['Katerino', 101038124], ['RocketBeansTV', 47627824], ['GubbaTV', 86198803]]
Usuario Katerino saiu. E seguia o Kandyland
Usuario Katerino saiu. E seguia o erobb221
SAIU
Leavers [['KendineMuzisyen', 79087140], ['wtcN', 51950404], ['JesusAVGN', 34711476], ['resttpowered', 50063424], ['erobb221', 96858382], ['melonyeahyeahyeah', 448404468], ['ThaldrinLol', 46595619], ['steel', 195675197], ['pqueen', 177249859], ['Dekarldent', 137907801], ['JLAmaru', 219431490], ['follentass', 230768385], ['Herr_Currywurst', 53886728]]
Usuario Herr_Currywurst saiu. E seguia o Kitboga
Usuario Dekarldent saiu. E seguia o Staiy
Usuario Herr_Currywurst saiu. E seguia o TobinatorLP
Usuario erobb221 saiu. E seguia o Kandyland
SAIU
Leavers [['alexelcapo', 36138196], ['Agrae

SAIU
Leavers [['DizzyKitten', 47474524], ['감블러', 130445097], ['꿀혜', 163469195], ['meishelly', 152764910], ['阿土', 26766376], ['JenaUrf', 54041313]]
Usuario 阿土 saiu. E seguia o 懶貓
Usuario 阿土 saiu. E seguia o 蝦愛橘子
Usuario 阿土 saiu. E seguia o 木棉花官方頻道
Usuario 阿土 saiu. E seguia o Jinnytty
SAIU
Leavers [['嬌兔', 15027034], ['Valeria7K', 192376620], ['Syrenia', 42036460], ['Sunlight', 35624606]]
SAIU
Leavers [['cyr', 37522866], ['天菜娘娘', 91099315], ['melharucos', 26819117], ['Ko0416', 100438963], ['peyzki', 25411785], ['문월님', 446766072]]
SAIU
Leavers [['jaomock', 39089007], ['쫀득이_', 134586700], ['윤이샘', 195247807], ['Hitch', 200186636]]
SAIU
Leavers [['pokimane', 44445592], ['러너', 149397653], ['大丸', 29546745], ['Igromania', 43534020], ['자동', 42848761], ['Everyeyeit', 52160263], ['초롱이', 102413421], ['Maghla', 131215608], ['수련수련', 419795496], ['peyzki', 25411785]]
SAIU
Leavers [['SBSmobile24', 423032920], ['짬타수아', 120568022], ['elded', 76385901], ['살인마협회장', 127718836], ['엔마_', 136236691]]
SAIU
Leave

Usuario Katerino saiu. E seguia o GoneDoc
SAIU
Leavers [['Agraelus', 36620767], ['TheMob', 186859486], ['김나성', 103991968], ['嬌兔', 15027034], ['MSFIIIRE', 235491613]]
Usuario Agraelus saiu. E seguia o Mizkif
SAIU
Leavers [['STPeach', 100484450], ['Surskity', 50465063], ['coelho', 56559621], ['VALECHOBERK', 125973046], ['Emarly', 73073047], ['SULAIMAN_iAi', 218812387]]
SAIU
Leavers [['QuarterJade', 173758090], ['velvet_7', 228169565]]
SAIU
Leavers [['Savjz', 43131877], ['PatoPapao', 35647075], ['1DrakoNz', 118787650], ['SungsLee', 183186817], ['고말숙_', 179811902]]
SAIU
Leavers [['FEDMYSTER', 39040630], ['PaymoneyWubby', 38251312], ['DizzyKitten', 47474524], ['Sykkuno', 26154978], ['청묘님', 176675885], ['IKeepItTaco', 57636576]]
SAIU
Leavers [['mayahiga', 235835559], ['개복어', 49233748], ['BruceGreene', 59250665], ['Koibu', 21654122], ['햇살살', 148842622], ['Caprimint', 59131475]]
SAIU
Leavers [['Mizkif', 94753024], ['CDNThe3rd', 14408894], ['ludwig', 40934651], ['우정잉', 150664679], ['JuanJuegaJu

Usuario jonzherka saiu. E seguia o djarii
Usuario Pettans saiu. E seguia o djarii
SAIU
Leavers [['TimTheTatman', 36769016], ['Locklear', 137347549], ['Tumblurr', 77827128], ['Staiy', 48865821], ['TSM_Viss', 90020006], ['에렌디라', 131912519], ['Vitecp', 17261176]]
Usuario Locklear saiu. E seguia o manuelferraratv
Usuario Locklear saiu. E seguia o theRyu7z
Usuario Staiy saiu. E seguia o DevinNash
SAIU
Leavers [['Sweet_Anita', 217377982], ['Kandyland', 57865494], ['Payo', 59624901], ['manuelferraratv', 131861345], ['STPeach', 100484450], ['Spt083', 116780430], ['B_U_L_O_C_H_K_A', 218301998], ['케인', 130433306], ['TaehaTypes', 73748917]]
SAIU
Leavers [['CurtisRyan', 70302631], ['memreyildiz', 192232160], ['flyinguwe87', 51492033], ['LotsOfBunnies', 81734181], ['HolaSoySergio1', 81795200], ['Yunicorn19', 130727794]]
SAIU
Leavers [['wtcN', 51950404], ['WELOVEGAMES', 30814134], ['YoDa', 47071880], ['Sweet_Anita', 217377982], ['H2P_Gucio', 36954803], ['BOBLEGOB', 89890241], ['Limmy', 10386664], ['

SAIU
Leavers [['Papaplatte', 50985620], ['JakenbakeLIVE', 11249217], ['과로사1', 439360392], ['HatFilms', 21945983], ['Yogscast', 20786541], ['resttpowered', 50063424], ['BeatPug', 134931057], ['TheBubbaArmy', 118342490]]
SAIU
Leavers [['TimTheTatman', 36769016], ['탬탬버린', 133923936], ['懶貓', 23852822], ['Lasqa', 60796327], ['박옥자누나', 160989002], ['Multiplayerit', 48199446], ['tenderlybae', 249559280], ['panetty', 132817946]]
Usuario Lasqa saiu. E seguia o NymN
Usuario 懶貓 saiu. E seguia o 木棉花官方頻道
Usuario 懶貓 saiu. E seguia o Ko0416
Usuario Lasqa saiu. E seguia o segall
SAIU
Leavers [['서새봄냥', 30904062], ['NymN', 62300805], ['PAGO3', 29468517], ['CeMka', 118263259], ['빢유환', 74138996], ['theoriginalmarkz', 250201188], ['The_Rock135', 92185138], ['JiNo_DesTroYeR', 45821367]]
Usuario NymN saiu. E seguia o Giantwaffle
Usuario NymN saiu. E seguia o HAchubby
Usuario NymN saiu. E seguia o Kandyland
Usuario NymN saiu. E seguia o OfficialAndyPyro
SAIU
Leavers [['EFEUYGAC', 103237625], ['Kandyland', 5786

,UserName,DeltaAbsolute,DeltaPercentage,ViewCount,TotalViewerCount,LeaversFollowers,RequestTime,Distance
0,NymN,1161,-0.041072,3565,5651854,0,2019-11-11 17:30:08,49
1,KingRichard,-6271,-12.094049,10667,31869562,1,2019-11-11 17:30:09,49
2,zoodasa,-291,-2.259014,4620,1609796,0,2019-11-11 17:30:09,49
3,견자희,-8,-0.988582,2378,10590705,1,2019-11-11 17:30:09,49
4,Homyatol,270,-0.364311,1691,2301857,1,2019-11-11 17:30:10,49
5,Staiy,1525,1.726047,358,20187035,2,2019-11-11 17:30:10,49
6,蝦愛橘子,-431,-1.427960,2183,18923183,0,2019-11-11 17:30:10,49
7,Kandyland,59,-0.623927,1692,10984482,0,2019-11-11 17:30:10,49
8,segall,-118,-0.811318,1619,5742055,0,2019-11-11 17:30:10,49
9,melonyeahyeahyeah,-245,-0.966097,1616,1252919,0,2019-11-11 17:30:10,49


In [9]:
dict_tt = links_dict(connection, "Teamfight Tactics")
tt_id = "513143"

time1 = datetime(2019, 11, 11, 17, 0) # 2019-11-11 17:00:02
time2 = datetime(2019, 11, 16, 4,30) # 2019-11-16 04:45:07

tt = get_progression(time1,time2,tt_id,dict_tt)
tt.to_csv("tt.csv")
tt

SAIU
Leavers [['IFresh_HD', 52616898], ['AlanzqTFT', 102845655], ['KingSobe', 61784358], ['mrs__honey', 140213270], ['MrStylet', 64100225], ['PokrovacHS', 41886240], ['TheFlyingRam', 58220468]]
Usuario IFresh_HD saiu. E seguia o Shaunz
Usuario IFresh_HD saiu. E seguia o TabzzHD
Usuario AlanzqTFT saiu. E seguia o TabzzHD
Usuario TheFlyingRam saiu. E seguia o TabzzHD
Usuario IFresh_HD saiu. E seguia o armatvhs
Usuario IFresh_HD saiu. E seguia o DarkHydra
Usuario AlanzqTFT saiu. E seguia o DarkHydra
Usuario TheFlyingRam saiu. E seguia o DarkHydra
Usuario IFresh_HD saiu. E seguia o 23x59
Usuario IFresh_HD saiu. E seguia o leduck_lol
Usuario AlanzqTFT saiu. E seguia o leduck_lol
Usuario IFresh_HD saiu. E seguia o bstln
Usuario TheFlyingRam saiu. E seguia o bstln
SAIU
Leavers [['Snowlz', 29468010], ['KenRhen', 82173521], ['bestsatojp', 54235016], ['Pottux', 28445597], ['Nastralia', 137955602], ['chenchen1771', 175437852]]
Usuario KenRhen saiu. E seguia o itsHafu
Usuario bestsatojp saiu. E se

SAIU
Leavers [['lolnani', 126846428], ['NiizumaStyle', 110226346], ['Betzzzz', 175828158]]
SAIU
Leavers [['Scarra', 22253819], ['keanelol', 74505272], ['AgonTFT', 55159684], ['힘의길', 222511449], ['AvolitePlays', 66053224], ['DesertMOOSE_', 98749280], ['edoode', 117151836], ['유나나_', 239053145], ['becajb', 161385960]]
SAIU
Leavers [['kyo1984123', 119497908], ['Lollash', 26408342], ['큐냥냥', 250992883], ['Gnom2D', 156836862], ['dalilachan', 426093780], ['iswimwithsharks', 205750813], ['Palaloa', 52028901]]
SAIU
Leavers [['Nick_Beckwith', 96701550], ['FP_SlapDaBass', 46437600], ['Faintmem', 142479305], ['Majuus', 9507567]]
SAIU
Leavers [['Pinch3HS', 44522508], ['miraclew0rker', 86440052], ['Veralion', 58571561], ['崔西兒', 29723716], ['OfficialSGTV', 41600380], ['Jyskgaming', 42550039]]
SAIU
Leavers [['嘎滋', 29130375], ['FujitoraHS', 76078255], ['米兒', 86091472]]
Usuario FujitoraHS saiu. E seguia o AlanzqTFT
SAIU
Leavers [['armatvhs', 142506321]]
SAIU
Leavers [['snoodyboo', 135629563], ['지수소녀', 10

SAIU
Leavers [['ernesBarbeQ', 41879247], ['KittyKathee', 78606566], ['LuckasTFT', 246993852], ['larissa', 76569551]]
SAIU
Leavers [['AKAWonder', 62951382], ['armatvhs', 142506321], ['Marples', 42373551], ['terror1989', 123359902], ['Glutth', 50891177]]
SAIU
Leavers [['Scarra', 22253819], ['Solary', 174955366], ['Shantao', 44098497], ['sushiBURRRito', 173995314], ['GraveyFries', 25742277]]
SAIU
Leavers [['ArQuel', 99681838], ['YamaChiika', 42443327], ['BALOTELLI777', 133577544], ['MMonte', 167323440], ['AskJoshy', 14379839], ['Keenly3', 93365335], ['XSamanthaXO93', 56210723]]
SAIU
Leavers [['뚜두1', 139967680], ['嘎滋', 29130375], ['Eillax', 50337055], ['Kronis_tv', 105163125]]
SAIU
Leavers [['Kurumx', 20795350], ['Macaiyla', 43126328], ['黑大大', 48657219], ['AndraxxusX', 11008415], ['RickiBabyyy', 126267978], ['IntoxiGames', 189884684], ['KrissiC9', 51200559]]
SAIU
Leavers [['정예지', 137472640], ['scarizardplays', 39657237], ['Nikki', 92640399], ['GFuelEnergy', 52795976], ['bunnycomb', 9108937

SAIU
Leavers [['Scarra', 22253819], ['Becca', 35824977], ['jschritte', 72960162], ['AKAWonder', 62951382], ['hyped', 14783735], ['뚜두1', 139967680], ['Picoca', 55947845], ['Zyrios7', 90684027], ['armatvhs', 142506321], ['Macaiyla', 43126328], ['aceofspadeshots', 54892328], ['BALOTELLI777', 133577544], ['DownToQuest', 151046563], ['SeePeeYou2', 456056219], ['Tholwmenos', 100415330], ['scarizardplays', 39657237], ['Johlee', 100841572], ['sunnylaplante', 185308021], ['Voglspinner', 174458469], ['shinopavels', 453236969], ['Marples', 42373551], ['LeonieMonty', 266398524], ['Nauru_', 65257142], ['Luden_Kuma', 452699906], ['Mariikawa', 96293897], ['MonteiroLombrado', 414650072], ['The_Viking_Monkey', 168698442], ['Chrystabel', 84757464], ['샛별아죠씨', 139312722], ['wondermelo', 152116430]]
SAIU
Leavers [['AKAWonder', 62951382], ['Picoca', 55947845], ['Miranda', 97557908], ['aceofspadeshots', 54892328], ['shinopavels', 453236969], ['Voglspinner', 174458469], ['LeonieMonty', 266398524], ['Chrystabe

SAIU
Leavers [['Your_Life_is_Mine_', 101350195], ['pimalive', 149250594], ['Patiizz', 141658755], ['EmmaSnorkel', 139124113], ['akichuuu', 81736783], ['Cooldown_tv', 100421608], ['ATeach', 208261930], ['Kee_tsune', 198717767]]
SAIU
Leavers [['luquelol', 88157480], ['RAFORO', 131750426], ['ReppukK', 174492294], ['D0L1K', 27687621], ['ZenMasTa', 192642697], ['TheJayBroski', 74319395]]
Usuario luquelol saiu. E seguia o TabzzHD
Usuario luquelol saiu. E seguia o DarkHydra
Usuario D0L1K saiu. E seguia o DarkHydra
SAIU
Leavers [['H2211Games', 28679933], ['Gwennlins', 250708304], ['Coxswain_Stroke', 228580846], ['OdinakaJesus', 14980250], ['kamil3e', 113279354], ['agresifporsuk', 118503396], ['EinfachAlu', 30768347], ['Madlencia', 39001921]]
Usuario H2211Games saiu. E seguia o TabzzHD
Usuario H2211Games saiu. E seguia o DarkHydra
SAIU
Leavers [['PennyArcade', 7443503], ['CoachPhaxe', 124969216], ['SaL_GG', 56912937]]
SAIU
Leavers [['jschritte', 72960162], ['DarkHydra', 23278143], ['idlemonkk',

SAIU
Leavers [['IFresh_HD', 52616898], ['Mathia', 59392726], ['박나나_', 184401308], ['NiMBuSCHiLL', 103263313], ['Angelixa', 91709944], ['Kimkou11', 42569015]]
Usuario IFresh_HD saiu. E seguia o leduck_lol
Usuario IFresh_HD saiu. E seguia o 23x59
SAIU
Leavers [['saintvicious', 30661033], ['superjj102', 58352686], ['Snowlz', 29468010], ['leduck_lol', 106899838], ['23x59', 231849482], ['idlemonkk', 455935663], ['LevelUpYourGame', 19482585], ['show__u', 174668816], ['peter82830', 37368014], ['ZhyTFT', 91339632], ['NautiBet', 109681654], ['寶豬', 114867910], ['Valarman', 84760442], ['chenchen1771', 175437852], ['MissHana', 466576505], ['Lollash', 26408342], ['Azzi_hots', 136828290]]
SAIU
Leavers [['歐蝟君', 47781604], ['RobetRogers', 97057430]]
SAIU
Leavers [['ToonTV', 116990597], ['Athena', 140551421], ['Nikolarn', 66272442], ['Weesie', 127957493]]
SAIU
Leavers [['Azzi_hots', 136828290]]
SAIU
Leavers [['tvgrypl', 19642462], ['BloumaxTR', 45883031], ['阿寧', 109593463], ['chu8', 30080810], ['armatv

,UserName,DeltaAbsolute,DeltaPercentage,ViewCount,TotalViewerCount,LeaversFollowers,RequestTime,Distance
0,Shaunz,-254,-2.617775,2796,5338276,1,2019-11-11 17:30:22,49
1,Sp4zie,197,0.665950,1650,25628699,0,2019-11-11 17:30:22,49
2,Evangelion0,528,3.082719,790,17244981,0,2019-11-11 17:30:22,49
3,TabzzHD,145,0.565724,1003,3237626,3,2019-11-11 17:30:22,49
4,ReventXz,117,0.377273,1031,4410832,0,2019-11-11 17:30:22,49
5,BlackFIreIce,-215,-1.835775,1303,25234271,0,2019-11-11 17:30:22,49
6,armatvhs,336,2.012727,360,47853103,1,2019-11-11 17:30:22,49
7,阿寧,-75,-0.748815,758,7407503,0,2019-11-11 17:30:23,49
8,DarkHydra,-15,-0.278534,512,408948,3,2019-11-11 17:30:23,49
9,Snowlz,15,-0.030888,354,2705510,0,2019-11-11 17:30:23,49





In [10]:
fm.linregress(pd.read_csv('cs.csv'), ['TotalViewerCount','ViewCount','LeaversFollowers','Distance'], 'DeltaPercentage')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        DeltaPercentage   R-squared:                       0.042
Model:                            OLS   Adj. R-squared:                  0.042
Method:                 Least Squares   F-statistic:                     103.3
Date:                Sat, 30 Nov 2019   Prob (F-statistic):           1.90e-65
Time:                        19:12:04   Log-Likelihood:                -15739.
No. Observations:                7035   AIC:                         3.149e+04
Df Residuals:                    7031   BIC:                         3.151e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
TotalViewerCount  1.792e-09   4.75e-10      3.772      0.000    8.61e-10    2.72e-09
ViewCount           -0.0001   1.37e-05     -9.186      0.000      -0.000   -9.87e-05
LeaversFollowers     1.9986      0.129     15.518      0.000       1.746       2.251
Distance             0.0027      0.001      4.609      0.000       0.002       0.004
==============================================================================
Omnibus:                    11282.765   Durbin-Watson:                   1.802
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         49432101.500
Skew:                           9.607   Prob(JB):                         0.00
Kurtosis:                     413.206   Cond. No.                     3.50e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 3.5e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [11]:
fm.linregress(pd.read_csv('lol.csv'), ['TotalViewerCount','ViewCount','LeaversFollowers','Distance'], 'DeltaPercentage')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        DeltaPercentage   R-squared:                       0.060
Model:                            OLS   Adj. R-squared:                  0.060
Method:                 Least Squares   F-statistic:                     186.7
Date:                Sat, 30 Nov 2019   Prob (F-statistic):          2.37e-117
Time:                        19:12:11   Log-Likelihood:                -14293.
No. Observations:                8715   AIC:                         2.859e+04
Df Residuals:                    8711   BIC:                         2.862e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
TotalViewerCount  6.425e-10   3.16e-10      2.033      0.042    2.28e-11    1.26e-09
ViewCount        -2.484e-06   4.15e-06     -0.598      0.550   -1.06e-05    5.66e-06
LeaversFollowers     0.6257      0.029     21.518      0.000       0.569       0.683
Distance            -0.0006      0.000     -1.764      0.078      -0.001    6.46e-05
==============================================================================
Omnibus:                    19906.933   Durbin-Watson:                   1.778
Prob(Omnibus):                  0.000   Jarque-Bera (JB):        717932541.243
Skew:                          21.128   Prob(JB):                         0.00
Kurtosis:                    1408.457   Cond. No.                     1.14e+08
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 1.14e+08. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [12]:
fm.linregress(pd.read_csv('tt.csv'), ['TotalViewerCount','ViewCount','LeaversFollowers','Distance'], 'DeltaPercentage')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        DeltaPercentage   R-squared:                       0.020
Model:                            OLS   Adj. R-squared:                  0.020
Method:                 Least Squares   F-statistic:                     41.42
Date:                Sat, 30 Nov 2019   Prob (F-statistic):           1.75e-26
Time:                        19:12:24   Log-Likelihood:                -12201.
No. Observations:                6024   AIC:                         2.441e+04
Df Residuals:                    6020   BIC:                         2.444e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
TotalViewerCount   2.15e-08   2.56e-09      8.403      0.000    1.65e-08    2.65e-08
ViewCount        -2.196e-05    3.9e-05     -0.563      0.573   -9.84e-05    5.45e-05
LeaversFollowers     0.4218      0.126      3.352      0.001       0.175       0.668
Distance             0.0013      0.001      2.553      0.011       0.000       0.002
==============================================================================
Omnibus:                     7499.344   Durbin-Watson:                   1.712
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         15825371.356
Skew:                           5.753   Prob(JB):                         0.00
Kurtosis:                     253.832   Cond. No.                     6.49e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 6.49e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
fm.linregress(pd.read_csv('jc.csv'), ['TotalViewerCount','ViewCount','LeaversFollowers','Distance'], 'DeltaPercentage')

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:        DeltaPercentage   R-squared:                       0.036
Model:                            OLS   Adj. R-squared:                  0.036
Method:                 Least Squares   F-statistic:                     95.70
Date:                Sat, 30 Nov 2019   Prob (F-statistic):           8.61e-61
Time:                        19:13:55   Log-Likelihood:                -11093.
No. Observations:                7583   AIC:                         2.219e+04
Df Residuals:                    7579   BIC:                         2.222e+04
Df Model:                           3                                         
Covariance Type:            nonrobust                                         
====================================================================================
                       coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------
TotalViewerCount  8.819e-09   7.53e-10     11.716      0.000    7.34e-09    1.03e-08
ViewCount         1.834e-05   6.02e-06      3.046      0.002    6.54e-06    3.01e-05
LeaversFollowers     0.3480      0.047      7.334      0.000       0.255       0.441
Distance             0.0009      0.000      2.871      0.004       0.000       0.001
==============================================================================
Omnibus:                     7769.541   Durbin-Watson:                   1.621
Prob(Omnibus):                  0.000   Jarque-Bera (JB):          3036144.205
Skew:                           4.388   Prob(JB):                         0.00
Kurtosis:                     100.633   Cond. No.                     7.53e+07
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.53e+07. This might indicate that there are
strong multicollinearity or other numerical problems.
"""